In [1]:
from datetime import datetime
import datetime as dt
import os
from operator import itemgetter
import shutil 
import pickle
import time
import json

from django.contrib.gis.geos.geometry import GEOSGeometry
from osgeo import osr
from pyproj import Proj
from fiona.crs import from_epsg
from fiona.crs import to_string
from sklearn.cluster import KMeans
import geopandas as gpd
from affine import Affine
import xarray as xr
import numpy as np
from rasterio.features import rasterize
from shapely.geometry import shape
import fiona
import glob
from dask.distributed import Client
from datacube.drivers.netcdf import write_dataset_to_netcdf
from datacube.api import GridWorkflow
import rasterio
from rasterio.mask import mask

from madmex.wrappers import gwf_query
from madmex.util.xarray import to_float, to_int
from madmex.models import PredictObject, TrainClassification
from madmex.models import Region

2020-02-23 13:46:44,818 - datacube.drivers.driver_cache - driver_cache safe_load: Failed to resolve driver datacube.plugins.index::s3aio_index
2020-02-23 13:46:44,820 - datacube.drivers.driver_cache - driver_cache safe_load: Error was: UndefinedEnvironmentName("'extra' does not exist in evaluation environment.",)
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:44292 remote=tcp://127.0.0.1:46703>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:44294 remote=tcp://127.0.0.1:46703>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:44296 remote=tcp://127.0.0.1:46703>
/home/madmex_user/.local/lib/python3.6/site-packages/distributed/worker.py:3269: UserWarning: Large object of size 25.02 MB detected in task graph: 
2020-02-23 15:06:29,394 - madmex.wrappers - loggerwriter write: /home/madmex_user/.local/lib/python3.6/site-packages/distributed/worker.py:3269: UserWarning: Large 

In [2]:
client = Client(n_workers=5,memory_limit='110GB', threads_per_worker=1)

In [3]:
client

Client Scheduler: tcp://127.0.0.1:46703 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 550.00 GB


In [4]:
client.restart()

Client Scheduler: tcp://127.0.0.1:46703 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 550.00 GB


In [5]:
region = 'cultivos_2019_simplify'
name_of_product = 's2_l2a_10m_scl_mexico'
begin = '2018-01-01'
end = '2019-12-31'
gwf_kwargs = {'region': region, 'product': name_of_product, 'begin': begin, 'end':end}
iterable = gwf_query(**gwf_kwargs)

In [6]:
lista_iter = list(iterable)

In [7]:
len(lista_iter)

53

In [8]:
[elem[0] for elem in lista_iter]

[(47, -35),
 (47, -37),
 (48, -37),
 (47, -36),
 (48, -36),
 (40, -31),
 (39, -31),
 (38, -30),
 (39, -30),
 (40, -30),
 (39, -29),
 (38, -29),
 (40, -29),
 (38, -28),
 (40, -28),
 (38, -27),
 (39, -28),
 (39, -27),
 (38, -26),
 (39, -26),
 (40, -33),
 (40, -32),
 (39, -32),
 (37, -29),
 (40, -34),
 (41, -33),
 (42, -33),
 (41, -32),
 (42, -32),
 (41, -31),
 (42, -31),
 (43, -33),
 (44, -34),
 (43, -35),
 (45, -34),
 (44, -35),
 (45, -37),
 (43, -34),
 (44, -37),
 (43, -36),
 (45, -36),
 (45, -35),
 (44, -36),
 (42, -34),
 (41, -34),
 (37, -27),
 (37, -26),
 (37, -28),
 (41, -30),
 (46, -35),
 (46, -34),
 (46, -36),
 (46, -37)]

# 1) Se elige el primer dc_tile de lista_iter para la generación de segmentos clusterizados

In [9]:
date_today = datetime.today().strftime("%m-%d-%Y")

In [10]:
index=49

In [11]:
var=lista_iter[index]

In [12]:
dc_tile = '%d_%d' % (var[0][0], var[0][1])

In [13]:
dc_tile

'46_-35'

In [14]:
date_today

'02-23-2020'

In [15]:
sr_0 = GridWorkflow.load(var[1],dask_chunks={'x': 1600, 'y': 1600,'time': 60})

In [16]:
sr_0.attrs['geobox'] = var[1].geobox

In [17]:
sr_0 = client.persist(sr_0)

**110gb is enough**

In [18]:
sr_0 = sr_0.apply(func=to_float, keep_attrs=True)

In [19]:
sr_1 = sr_0.where(sr_0.pixel_qa.isin([2,4,5,6,7,8,11]))

In [20]:
sr_1['ndvi'] = ((sr_1.nir - sr_1.red) / (sr_1.nir + sr_1.red)) * 10000

In [21]:
sr_1['ndvi'].attrs['nodata'] = 0

In [22]:
sr_1['gndvi'] = ((sr_1.nir - sr_1.green) / (sr_1.nir + sr_1.green)) * 10000

In [23]:
sr_1['gndvi'].attrs['nodata'] = 0

In [24]:
sr_1['avi'] = ((sr_1.nir*(1-sr_1.red))*(sr_1.nir-sr_1.red))**(1/3)

In [25]:
sr_1['avi'].attrs['nodata'] = 0

In [26]:
sr_1 = client.persist(sr_1)

# 2) Cálculo de features

In [27]:
ndvi_mean_resampled = sr_1.ndvi.resample(time='M').mean()

In [28]:
ndvi_mean_resampled = client.persist(ndvi_mean_resampled)

In [29]:
ndvi_mean_resampled_drop_na = ndvi_mean_resampled.dropna('time',how='all')

In [30]:
gndvi_mean_resampled = sr_1.gndvi.resample(time='M').mean()

In [31]:
gndvi_mean_resampled = client.persist(gndvi_mean_resampled)

In [32]:
gndvi_mean_resampled_drop_na = gndvi_mean_resampled.dropna('time',how='all')

In [33]:
avi_mean_resampled = sr_1.avi.resample(time='M').mean()

In [34]:
avi_mean_resampled = client.persist(avi_mean_resampled)

In [35]:
avi_mean_resampled_drop_na = avi_mean_resampled.dropna('time',how='all')

In [36]:
combined = xr.merge([ndvi_mean_resampled_drop_na,
                     gndvi_mean_resampled_drop_na,
                     avi_mean_resampled_drop_na])

In [37]:
combined = client.persist(combined)

In [38]:
features = list(combined.data_vars)
s = '-'
features_string = s.join(features)

In [39]:
path_result = '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/' + features_string + '/'

In [40]:
path_result_dc_tile = os.path.join(path_result, date_today 
                                   + '/' + dc_tile + '/')

In [41]:
path_result_dc_tile

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-23-2020/46_-35/'

In [42]:
if not os.path.exists(path_result_dc_tile):
    os.makedirs(path_result_dc_tile)

In [43]:
nc_filename = path_result_dc_tile + date_today + '_combined_' + dc_tile + '.nc'

In [44]:
combined.attrs = sr_0.attrs

In [45]:
combined.coords['time'].attrs = sr_0.coords['time'].attrs
combined.coords['x'].attrs = sr_0.coords['x'].attrs
combined.coords['y'].attrs = sr_0.coords['y'].attrs

In [46]:
#write_dataset_to_netcdf(combined.compute(),nc_filename)

# 3) Leer segmentación

In [47]:
geom = GEOSGeometry(json.dumps(sr_0.geobox.geographic_extent.json))

In [48]:
seg_name = 'seg_mex_s2_10m_scl_2018_2019'

In [49]:
qs = PredictObject.objects.filter(the_geom__contained=geom,segmentation_information__name=seg_name)

In [50]:
path_seg = qs[0].path

In [51]:
with fiona.open(path_seg) as src:
    fc = list(src)
    crs = to_string(src.crs)

In [52]:
fc_subset=((x['properties']['id'], x['geometry']) for x in fc)

In [53]:
fc_sorted = sorted(fc_subset, key=itemgetter(0))

**First position of fc_sorted is id, second position is geometry**


In [54]:
iterable_zip = zip([k[1] for k in fc_sorted], [k[0] for k in fc_sorted])

In [55]:
aff = Affine(*list(sr_0.affine)[0:6])

In [56]:
dimensions_dataset = list(sr_0.coords)
list_dimensions = [x for x in dimensions_dataset if x != 'time']
lambda_function = lambda l_netcdf,l_test: l_netcdf[0] if l_netcdf[0] in l_test else l_netcdf[1]
xdim = lambda_function(list_dimensions,['x','longitude'])
ydim = lambda_function(list_dimensions,['y','latitude'])

In [57]:
fc_raster = rasterize(iterable_zip, transform=aff,
                      out_shape=(sr_0.sizes[ydim], sr_0.sizes[xdim]),
                      dtype='float64', fill=np.nan)

In [58]:
fc_raster = fc_raster.astype(int)

In [59]:
fc_dataarray = xr.DataArray(fc_raster, dims=[ydim, xdim], name='features_id')

# 4) Construcción de dataframes para pasar al kmeans

In [60]:
mask_file='/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/madmex_masks_agricultura_pastizal/madmex_mascara_s2_2018_agricultura_pastizal_Veracruz_una_clase.tif'

Abrir máscara, recortar máscara a tamaño de extent de dc tile

In [61]:
crs_dict = {'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'a':6378137,
 'b':6378136.027241431,
 'wktext': True}

#+ellps=WGS84 +datum=WGS84

In [72]:
geometry_extent = sr_0.geobox.extent.json

In [92]:
with rasterio.open(mask_file) as mask_src:
    mask_src_crop, out_transform = mask(mask_src, [geometry_extent], crop=True, filled=True)
    mask_meta = mask_src.meta.copy()

In [93]:
mask_meta

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': 0.0,
 'width': 55197,
 'height': 56935,
 'count': 1,
 'crs': CRS.from_wkt('PROJCS["Lambert_Conformal_Conic",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'),
 'transform': Affine(10.000044382839143, 0.0, 2839949.1166991508,
        0.0, -10.000071023378611, 1166185.8850332226)}

In [94]:
mask_src_crop.shape

(1, 5003, 5003)

In [95]:
out_transform

Affine(10.000044382839143, 0.0, 3278071.061200099,
       0.0, -10.000071023378611, 726042.7590102365)

In [96]:
combined.ndvi.shape

(12, 5002, 5002)

In [97]:
mask_meta.update(width=combined.ndvi.shape[2],
                 height=combined.ndvi.shape[1],
                 transform=out_transform,
                 compress='lzw')

In [98]:
mask_meta

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': 0.0,
 'width': 5002,
 'height': 5002,
 'count': 1,
 'crs': CRS.from_wkt('PROJCS["Lambert_Conformal_Conic",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'),
 'transform': Affine(10.000044382839143, 0.0, 3278071.061200099,
        0.0, -10.000071023378611, 726042.7590102365),
 'compress': 'lzw'}

In [99]:
mask_crop_file='/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/madmex_masks_agricultura_pastizal/madmex_mascara_s2_2018_agricultura_pastizal_Veracruz_una_clase_crop.tif'

In [100]:
with rasterio.open(mask_crop_file,'w',**mask_meta) as dst:
    dst.write(mask_src_crop)
    

In [101]:
mask_src_crop = rasterio.open(mask_crop_file)

In [102]:
mask_src_crop.shape

(5002, 5002)

In [106]:
with rasterio.open(mask_crop_file,'r') as src:
    array = src.read()
    combined_masked = combined.where(array==1)

In [107]:
combined_masked

<xarray.Dataset>
Dimensions:  (time: 12, x: 5002, y: 5002)
Coordinates:
  * time     (time) datetime64[ns] 2018-10-31 2018-11-30 ... 2019-09-30
  * y        (y) float64 7.26e+05 7.26e+05 7.26e+05 ... 6.76e+05 6.76e+05
  * x        (x) float64 3.278e+06 3.278e+06 3.278e+06 ... 3.328e+06 3.328e+06
Data variables:
    ndvi     (time, y, x) float64 dask.array<chunksize=(1, 1600, 1600), meta=np.ndarray>
    gndvi    (time, y, x) float64 dask.array<chunksize=(1, 1600, 1600), meta=np.ndarray>
    avi      (time, y, x) float64 dask.array<chunksize=(1, 1600, 1600), meta=np.ndarray>
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...
    geobox:   GeoBox(Geometry({'type': 'Polygon', 'coordinates': [[(-94.62445...

In [108]:
combined = xr.merge([fc_dataarray,combined_masked])

In [109]:
combined = client.persist(combined)

In [110]:
combined = combined.stack(z=(xdim, ydim))
combined = combined.reset_index('z')
combined = combined.drop([xdim,ydim])

In [111]:
combined

<xarray.Dataset>
Dimensions:      (time: 12, z: 25020004)
Coordinates:
  * time         (time) datetime64[ns] 2018-10-31 2018-11-30 ... 2019-09-30
Dimensions without coordinates: z
Data variables:
    features_id  (z) int64 464931 464931 464931 464931 ... 823 823 823 824
    ndvi         (time, z) float64 dask.array<chunksize=(1, 2000800), meta=np.ndarray>
    gndvi        (time, z) float64 dask.array<chunksize=(1, 2000800), meta=np.ndarray>
    avi          (time, z) float64 dask.array<chunksize=(1, 2000800), meta=np.ndarray>

In [112]:
combined = combined.where((np.isfinite(combined['features_id'])), drop=True)

In [113]:
combined = client.persist(combined)

In [114]:
df = combined.to_dataframe()

In [115]:
df.head()

features_id  ndvi  gndvi  avi
time       z                               
2018-10-31 0     464931.0   NaN    NaN  NaN
           1     464931.0   NaN    NaN  NaN
           2     464931.0   NaN    NaN  NaN
           3     464931.0   NaN    NaN  NaN
           4     464931.0   NaN    NaN  NaN

In [116]:
del sr_0
del ndvi_mean_resampled_drop_na
del gndvi_mean_resampled_drop_na
del avi_mean_resampled_drop_na

In [117]:
client.restart()

Client Scheduler: tcp://127.0.0.1:46703 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 550.00 GB


In [118]:
df=df.reset_index('time')

In [119]:
df2 = df.groupby(['time','features_id'],as_index=False).agg({features[0]: 'median',
                                                             features[1]: 'median',
                                                             features[2]: 'median'})

In [120]:
df2.index = df2['time']

In [121]:
df2.drop(['time'], axis=1, inplace=True)

In [122]:
#df_filename = path_result_dc_tile + date_today + '_df' + '.csv'

In [123]:
#df2.to_csv(df_filename)

In [124]:
times_df = [datetime.date(d).strftime('%Y-%m-%d') for d in df2.index.get_level_values('time').unique()]

In [125]:
df3 = df2.loc[times_df[0]]

In [126]:
df3.rename(columns={features[0]: features[0]+'_' + times_df[0],
                    features[1]: features[1]+'_' +times_df[0],
                    features[2]: features[2]+'_' +times_df[0]},
                 inplace=True)

In [127]:
df3=df3.reset_index('time')

In [128]:
df3.drop(['time'],axis=1,inplace=True)

In [129]:
for k in range(1,len(times_df)):
    df_subset = df2.loc[times_df[k]]
    df_subset.dropna(how='all', inplace=True)
    df_subset.rename(columns={features[0]: features[0]+'_' + times_df[k],
                        features[1]: features[1]+'_' +times_df[k],
                        features[2]: features[2]+'_' +times_df[k]},
                     inplace=True)
    df_subset=df_subset.reset_index('time')
    df_subset.drop(['time'],axis=1,inplace=True)
    df3 = df3.join(df_subset.set_index('features_id'), on='features_id')

In [132]:
len(df3.index)

465190

In [133]:
sum(df3['ndvi_2018-10-31'].isna())

223893

In [134]:
df3.head()

,features_id,ndvi_2018-10-31,gndvi_2018-10-31,avi_2018-10-31,ndvi_2018-11-30,gndvi_2018-11-30,avi_2018-11-30,ndvi_2018-12-31,gndvi_2018-12-31,avi_2018-12-31,...,avi_2019-06-30,ndvi_2019-07-31,gndvi_2019-07-31,avi_2019-07-31,ndvi_2019-08-31,gndvi_2019-08-31,avi_2019-08-31,ndvi_2019-09-30,gndvi_2019-09-30,avi_2019-09-30
0,0.0,NaN,NaN,NaN,3037.542662,3519.637462,NaN,6343.367305,5714.586237,NaN,...,NaN,7072.310406,5471.497070,NaN,5927.507849,4976.339159,157.620844,5429.192322,4179.704962,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,6808.742572,5992.518173,NaN,...,NaN,7479.338843,5785.714286,NaN,7422.893401,6102.539867,NaN,7720.264317,6205.787781,NaN
2,2.0,NaN,NaN,NaN,1304.347826,995.024876,NaN,5277.256033,5073.044689,NaN,...,NaN,4610.591900,3387.998509,NaN,4065.996744,3805.391521,228.563695,3458.110517,2821.782178,NaN
3,3.0,NaN,NaN,NaN,2188.139059,1153.846154,NaN,4025.697454,4437.935006,NaN,...,NaN,4116.343490,3206.550449,NaN,4808.338245,4256.782439,224.011213,4071.111111,4000.000000,NaN
4,4.0,NaN,NaN,NaN,303.984260,-1259.740260,50.211372,3911.114125,4364.779671,NaN,...,NaN,4272.901895,3313.120526,NaN,1836.229809,1785.441479,230.654221,2457.778502,1560.645245,NaN


**Join df with fc in geopandas dataframe form**

In [427]:
#crs_dict = {'proj': 'lcc',
# 'lat_1': 17.5,
# 'lat_2': 29.5,
# 'lat_0': 12,
# 'lon_0': -102,
# 'x_0': 2500000,
# 'y_0': 0,
# 'datum': 'WGS84',
# 'units': 'm',
# 'no_defs': True,
# 'wktext': True}

In [135]:
gdf = gpd.GeoDataFrame.from_features([feature for feature in fc], 
                                      crs = crs_dict)


In [136]:
gdf = gdf.rename(columns={'id':'features_id'})

In [137]:
gdf_df_join = gdf.merge(df3, on='features_id', how='left')

In [171]:
gdf_df_join.head()

,geometry,features_id,ndvi_2018-10-31,gndvi_2018-10-31,avi_2018-10-31,ndvi_2018-11-30,gndvi_2018-11-30,avi_2018-11-30,ndvi_2018-12-31,gndvi_2018-12-31,...,avi_2019-06-30,ndvi_2019-07-31,gndvi_2019-07-31,avi_2019-07-31,ndvi_2019-08-31,gndvi_2019-08-31,avi_2019-08-31,ndvi_2019-09-30,gndvi_2019-09-30,avi_2019-09-30
0,"POLYGON ((3278820.000 726040.000, 3278860.000 ...",465139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"POLYGON ((3282730.000 726040.000, 3282770.000 ...",465141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"POLYGON ((3282800.000 726040.000, 3282810.000 ...",465172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"POLYGON ((3282910.000 726040.000, 3282930.000 ...",465184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"POLYGON ((3282990.000 726040.000, 3283050.000 ...",465158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
len(gdf_df_join.index)

465190

In [173]:
sum(gdf_df_join['ndvi_2018-10-31'].isna())

223893

In [177]:
features_string

'ndvi-gndvi-avi'

In [183]:
gdf_df_join_reduced = gdf_df_join.drop(['features_id','geometry'],axis=1)

In [184]:
gdf_df_join_reduced.head()

,ndvi_2018-10-31,gndvi_2018-10-31,avi_2018-10-31,ndvi_2018-11-30,gndvi_2018-11-30,avi_2018-11-30,ndvi_2018-12-31,gndvi_2018-12-31,avi_2018-12-31,ndvi_2019-01-31,...,avi_2019-06-30,ndvi_2019-07-31,gndvi_2019-07-31,avi_2019-07-31,ndvi_2019-08-31,gndvi_2019-08-31,avi_2019-08-31,ndvi_2019-09-30,gndvi_2019-09-30,avi_2019-09-30
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
gdf_df_join_reduced.dropna(how='all', inplace=True)

In [186]:
gdf_df_join_reduced.head()

,ndvi_2018-10-31,gndvi_2018-10-31,avi_2018-10-31,ndvi_2018-11-30,gndvi_2018-11-30,avi_2018-11-30,ndvi_2018-12-31,gndvi_2018-12-31,avi_2018-12-31,ndvi_2019-01-31,...,avi_2019-06-30,ndvi_2019-07-31,gndvi_2019-07-31,avi_2019-07-31,ndvi_2019-08-31,gndvi_2019-08-31,avi_2019-08-31,ndvi_2019-09-30,gndvi_2019-09-30,avi_2019-09-30
17600,8129.629630,6846.633685,NaN,8263.767456,7061.840333,NaN,6982.468222,6867.747371,NaN,6602.215707,...,NaN,7413.689362,6348.699616,NaN,7043.735261,6397.396399,NaN,6762.012304,6237.153407,NaN
17641,8273.310311,6780.155642,NaN,7667.744749,6601.027059,NaN,4691.556129,5344.083626,NaN,5997.543272,...,NaN,7396.603106,6389.116977,NaN,7065.971270,6321.946850,NaN,6872.804049,6222.021285,NaN
17703,8683.057148,7339.715586,NaN,8682.902671,7409.228412,NaN,8780.981837,7767.202899,NaN,8130.762803,...,NaN,8369.722623,7170.007035,NaN,8704.088762,7413.837170,NaN,8780.899978,7368.312722,NaN
17711,8626.259456,7291.981467,NaN,8343.389792,7168.537785,NaN,8549.297765,7603.481660,NaN,7793.991630,...,NaN,7647.384620,6599.373017,NaN,8184.705604,7043.405821,NaN,8216.754528,7068.512543,NaN
17740,8442.433834,7272.651701,NaN,8425.589021,7306.261323,NaN,8218.254596,7393.339403,NaN,7491.073316,...,NaN,7923.047856,6774.253901,NaN,7803.778356,6826.393754,NaN,7724.370093,6837.747611,NaN


In [192]:
len(gdf_df_join_reduced.index)

287078

In [190]:
t_keep_rows = gdf_df_join_reduced.index.values

In [193]:
gdf_df_join_reduced2 = gdf_df_join.loc[gdf_df_join.index.isin(t_keep_rows)]

In [194]:
gdf_df_join_reduced2.head()

,geometry,features_id,ndvi_2018-10-31,gndvi_2018-10-31,avi_2018-10-31,ndvi_2018-11-30,gndvi_2018-11-30,avi_2018-11-30,ndvi_2018-12-31,gndvi_2018-12-31,...,avi_2019-06-30,ndvi_2019-07-31,gndvi_2019-07-31,avi_2019-07-31,ndvi_2019-08-31,gndvi_2019-08-31,avi_2019-08-31,ndvi_2019-09-30,gndvi_2019-09-30,avi_2019-09-30
17600,"POLYGON ((3278130.000 719750.000, 3278140.000 ...",447535,8129.629630,6846.633685,NaN,8263.767456,7061.840333,NaN,6982.468222,6867.747371,...,NaN,7413.689362,6348.699616,NaN,7043.735261,6397.396399,NaN,6762.012304,6237.153407,NaN
17641,"POLYGON ((3278130.000 719720.000, 3278170.000 ...",447536,8273.310311,6780.155642,NaN,7667.744749,6601.027059,NaN,4691.556129,5344.083626,...,NaN,7396.603106,6389.116977,NaN,7065.971270,6321.946850,NaN,6872.804049,6222.021285,NaN
17703,"POLYGON ((3278240.000 719730.000, 3278280.000 ...",447410,8683.057148,7339.715586,NaN,8682.902671,7409.228412,NaN,8780.981837,7767.202899,...,NaN,8369.722623,7170.007035,NaN,8704.088762,7413.837170,NaN,8780.899978,7368.312722,NaN
17711,"POLYGON ((3278280.000 719720.000, 3278300.000 ...",447461,8626.259456,7291.981467,NaN,8343.389792,7168.537785,NaN,8549.297765,7603.481660,...,NaN,7647.384620,6599.373017,NaN,8184.705604,7043.405821,NaN,8216.754528,7068.512543,NaN
17740,"POLYGON ((3278080.000 719800.000, 3278100.000 ...",447379,8442.433834,7272.651701,NaN,8425.589021,7306.261323,NaN,8218.254596,7393.339403,...,NaN,7923.047856,6774.253901,NaN,7803.778356,6826.393754,NaN,7724.370093,6837.747611,NaN


# 5) Unsupervised clustering with Kmeans

**Número de clústers depende del número de cultivos presentes en el dc_tile** 

**Revisar credenciales de db**

In [195]:
var[1].geobox.geographic_extent.json

{'type': 'Polygon',
 'coordinates': [[(-94.62445901386619, 18.307292421720017),
   (-94.64874005892365, 17.85809774251316),
   (-94.1770621971074, 17.834324723281572),
   (-94.15122923779683, 18.283423396251024),
   (-94.62445901386619, 18.307292421720017)]]}

In [196]:
training_set = 'cultivos2019_inecol'

In [197]:
qs_t = TrainClassification.objects.filter(train_object__the_geom__contained=geom,
                                        training_set=training_set).prefetch_related('interpret_tag')

In [198]:
l_tag_ids = []

In [199]:
for element in qs_t:
    l_tag_ids.append(element.interpret_tag.id)

In [200]:
nclusters = len(np.unique(l_tag_ids))

In [201]:
nclusters

7

In [202]:
kmeans = KMeans(n_clusters=nclusters, n_jobs=10)

In [203]:
features_for_kmeans = [column for column in list(gdf_df_join_reduced2.columns) if column not in ['geometry','features_id']]

In [204]:
features_for_kmeans

['ndvi_2018-10-31',
 'gndvi_2018-10-31',
 'avi_2018-10-31',
 'ndvi_2018-11-30',
 'gndvi_2018-11-30',
 'avi_2018-11-30',
 'ndvi_2018-12-31',
 'gndvi_2018-12-31',
 'avi_2018-12-31',
 'ndvi_2019-01-31',
 'gndvi_2019-01-31',
 'avi_2019-01-31',
 'ndvi_2019-02-28',
 'gndvi_2019-02-28',
 'avi_2019-02-28',
 'ndvi_2019-03-31',
 'gndvi_2019-03-31',
 'avi_2019-03-31',
 'ndvi_2019-04-30',
 'gndvi_2019-04-30',
 'avi_2019-04-30',
 'ndvi_2019-05-31',
 'gndvi_2019-05-31',
 'avi_2019-05-31',
 'ndvi_2019-06-30',
 'gndvi_2019-06-30',
 'avi_2019-06-30',
 'ndvi_2019-07-31',
 'gndvi_2019-07-31',
 'avi_2019-07-31',
 'ndvi_2019-08-31',
 'gndvi_2019-08-31',
 'avi_2019-08-31',
 'ndvi_2019-09-30',
 'gndvi_2019-09-30',
 'avi_2019-09-30']

In [206]:
kmeans.fit(gdf_df_join_reduced2[features_for_kmeans].fillna(0).values)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=7, n_init=10, n_jobs=10, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

**Write model fit**

In [207]:
model_filename = path_result_dc_tile + date_today + '_model_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi' + '.pkl'

In [208]:
model_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-23-2020/46_-35/02-23-2020_model_46_-35_nclusters_7_aoi.pkl'

In [209]:
pickle.dump(kmeans, open(model_filename, "wb"))

**Predict with model**

In [210]:
model_filename = path_result_dc_tile + date_today + '_model_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi' + '.pkl'

In [211]:
model_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-23-2020/46_-35/02-23-2020_model_46_-35_nclusters_7_aoi.pkl'

In [212]:
kmeans = pickle.load(open(model_filename, "rb"))

**Predict**

In [213]:
preds = kmeans.predict(gdf_df_join_reduced2[features_for_kmeans].fillna(0).values)

In [214]:
gdf_df_clusters = gdf_df_join_reduced2.drop(features_for_kmeans,axis=1)


In [215]:
gdf_df_clusters['preds'] = preds

In [216]:
gdf_df_clusters.head()

,geometry,features_id,preds
17600,"POLYGON ((3278130.000 719750.000, 3278140.000 ...",447535,3
17641,"POLYGON ((3278130.000 719720.000, 3278170.000 ...",447536,3
17703,"POLYGON ((3278240.000 719730.000, 3278280.000 ...",447410,3
17711,"POLYGON ((3278280.000 719720.000, 3278300.000 ...",447461,3
17740,"POLYGON ((3278080.000 719800.000, 3278100.000 ...",447379,3


# 6) Write results to file

**Geopackage**

In [217]:
path_results_geopackage = '/home/madmex_user/agriculture_geopackages/features_computed/' + features_string + '/'

In [218]:
path_result_geopackage = os.path.join(path_results_geopackage, date_today 
                                       + '/' + dc_tile + '/')

In [219]:
if not os.path.exists(path_result_geopackage):
    os.makedirs(path_result_geopackage)

In [220]:
clusters_filename = path_result_geopackage + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi2' + '.gpkg'

In [221]:
layer_name = date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi'

In [222]:
gdf_df_clusters.crs = crs_dict

In [223]:
gdf_df_clusters.to_file(clusters_filename,
                        layer=layer_name,
                        driver='GPKG')

In [224]:
shutil.move(clusters_filename,path_result_dc_tile)

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-23-2020/46_-35/02-23-2020_clusters_46_-35_nclusters_7_aoi2.gpkg'

**Shapefile**

In [225]:
clusters_filename = path_result_dc_tile + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi2' + '.shp'

In [226]:
gdf_df_clusters.to_file(clusters_filename)


In [227]:
clusters_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-23-2020/46_-35/02-23-2020_clusters_46_-35_nclusters_7_aoi2.shp'